### This purpose of this code is to read raw altimetry data from a server in Alaska and then ingest it into the postgres database

#### we'll use the Python library http://pysftp.readthedocs.org/en/release_0.2.8/

#### A. Arendt, S. Nimmagadda, C. Larsen

##### rev. 2015 11 17: initial creation
##### 2015 11 29: stepping through ReadLambFile from Evan's UpdateDb.py; removing str/dict distinction; trying in Pandas instead

establish the secure FTP connection:

In [1]:
%matplotlib inline 
import pandas as pd
import pysftp
import Altimetry as alt
import UpdateDb as udb
from base64 import b64decode as readpassword
from sqlalchemy import create_engine

user = 'sai'
word = 'dGVzdA=='

engine = create_engine('postgresql://' + user + ':' + readpassword(word) + '@localhost:5432/sandbox')

In [6]:
# just do this once
engine.execute("CREATE TABLE lambtest (gid serial PRIMARY KEY, rgiid character varying(14), date1 date, date2 date, \
               interval smallint, volmodel real, vol25diff real,vol75diff real, balmodel real, bal25diff real, \
               bal75diff real,e integer[],dz real[],dz25 real[],dz75 real[],aad real[],masschange real[], \
               massbal real[],numdata integer[]);")

In [7]:
host = readpassword('YmFpcmQuZ3BzLmFsYXNrYS5lZHU=')
user = readpassword('bGFzZXI=')
pw = readpassword('MjYpKSlrbQ==')
sftp = pysftp.Connection(host, username=user, password=pw)

notInRGI = []
with sftp.cd('/home/laser/analysis/'):
    folderList = sftp.listdir()
    for glacierName in folderList:
        print(glacierName)
        if sftp.exists(glacierName + '/results'):
            with (sftp.cd(glacierName + '/results/')):
                for fileName in sftp.listdir():
                    if fileName.endswith(".output.txt"):
                        sftp.get(fileName)
                        query = "SELECT rgiid from modern WHERE name LIKE '" + glacierName + "%'"
                        try:
                            rgiid = str(alt.GetSqlData(query)['rgiid'][0])
                        except:
                            notInRGI.append(glacierName)
                        sql = udb.lamb_sql_generator(fileName, rgiid, 'lambtest')
                        engine.execute(sql)


.DS_Store
._Analysis_List.txt
._list.txt
.directory
.gmtcommands4
.gmtdefaults4
.shp.zone.xy
2009
2010
Aialik
Allen
Alsek
Analysis_List.txt
ArtLewis
BagleyEast_old
BagleySystem
BagleyWest_old
Bainbridge
Baird
Barnard
Battle
Bear
BearLake
Bench
Bering
Bering_old
Bering_reallyold
BlackRapids
Blockade
Brady
Canwell
Carroll
Casement
ChejaSouth
ChejaWest
Chenega
Chernof
ChigmitNorth
ChigmitSouth
Chisana
Chitina
Colony
Columbia
Copper
Crillon
Dall
Davidson
Dawes
Deserted
Dinglestadt
Donjek
Double
DoubleNorth
DoubleSouth
Eagle
EastFork
EastNunatak
Eklutna
Eldridge
Ellsworth
Excelsior
Exit
Fairweather
Falling
Ferris
Field
Flood
Gerstle
GilkeyBucher
Gillam
GlacierBay
GrandPacific
GrandPlateau
Great
Grotto
Gulkana
Guyot
Harding
Harris
Harvard
Hayes
Hidden
Holgate
Hubbard
INSTRUCTIONS
Jefferies_old
JohnsHopkins
Johnson
Kachemak
Kahiltna
Kaskawulsh
Kennicott
Killey
Klutlan
Knik
Konamoxt
LaPerouse
Lacuna
Lamplugh
LeConte
LemonCreek
LittleDinglestadt
LittleJarvis
Lituya
Llewellyn
Logan
Lowell
MacLar

In [10]:
t = list(set(notInRGI))
print(str(t))

[u'Rohn', u'BearLake', u'WestNunatak', u'EastNunatak', u'YakutatWest', u'GilkeyBucher', u'WestFork', u'Turquoise', u'GrandPlateau', u'Malaspina', u'WestGulkana', u'TsinaSouth', u'NorthDawes', u'EastFork', u'ArtLewis', u'MacLaren', u'NorthBaird', u'LemonCreek', u'YakutatEast', u'LittleJarvis', u'GrandPacific', u'ReynoldsSouth', u'Ogilvie', u'MarcusBaker', u'Northeastern', u'VernRitchie', u'SouthSawyer', u'BlackRapids', u'WarmCreek']


In [ ]:
import Altimetry as alt
%matplotlib inline
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
surveyeddata = alt.GetLambData(verbose=False,longest_interval=True,interval_max=30,interval_min=5,by_column=True,
                           as_object=True)
surveyeddata.fix_terminus()
surveyeddata.normalize_elevation()
surveyeddata.calc_dz_stats()

#Now to partition the dataset as done in Larsen etal by glacier type.
#We excluding surgers and outlier glaciers because we don't want those glaciers to affect 
#   mean profiles used for extrapolation.
types = ["gltype=0","gltype=1","gltype=2"]
lamb,userwheres,notused,whereswo,notswo = alt.partition_dataset(types,applytoall=["surge='f'","name NOT IN ('Columbia Glacier','West Yakutat Glacier','East Yakutat Glacier')"])
results = alt.extrapolate('testing',lamb,whereswo,insert_surveyed_data=surveyeddata,keep_postgres_tbls=False) 

In [ ]:
import numpy as N
#Plotting a pie chart of the total amount of mass lost from each glacier type.
typenames = {'0':'Land','1':'Tidewater','2':'Lake'}
labels = [typenames[i] for i in results['bytype']['gltype'].astype(str)]
figure(figsize=(3,3))
pi = pie(N.abs(results['bytype']['totalgt']),labels=labels)

In [ ]:
results['bytype']['totalgt'].sum()